#**Машинное обучение ИБ-2024**

#**Домашнее задание 2.**
#Классификация, KNN, LogReg, SVC.

In [1]:
from typing import Tuple, List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline

sns.set(style="darkgrid")

## **Теоретическая Часть**

Мы рассматриваем задачу бинарной классификации. Для прогнозирования мы хотели бы использовать модель логистической регрессии. Для регуляризации мы добавляем комбинацию штрафов в размере $l_2$ и $l_1$ (Elastic Net).

Каждый объект в обучающем наборе данных индексируется с помощью $i$ и описывается парой: объекты $x_i\in\mathbb{R}^{K}$ и двоичные метки $y_i$. Модель параметризуется со смещением $w_0\in\mathbb{R}$ и весами $w\in\mathbb{R}^K$.

Задача оптимизации в отношении $w_0, w$ заключается в следующем (Elastic Net Loss):

$$L(w, w_0) = \frac{1}{N} \sum_{i=1}^N \ln(1+\exp(-y_i(w^\top x_i+w_0))) + \gamma \|w\|_1 + \beta \|w\|_2^2$$.



Градиенты функции потерь логистической регрессии представлены ниже:

$$dL(w, w_0)/ dw = -\frac{1}{N}  \frac{X*y^\top}{1 + \exp(y (Xw+w_0)))} + \gamma * sign(w) + 2 * beta * w$$

$$dL(w, w_0)/ dw_0 = -\frac{1}{N}  \frac{y}{1 + \exp(y*(Xw+w_0)))}$$

#### 1. [0.5 Балл] Реализуйте функцию, выдающий значение функции потерь логичтической регрессии:

In [8]:
def loss(X, y, w: List[float], w0: float, gamma=1., beta=1.) -> float:
    N = len(y)
    w = np.array(w)
    linear_comb = X.dot(w) + w0
    log_loss = np.log(1 + np.exp(-y * linear_comb))
    avg_log_loss = np.mean(log_loss)
    regularization = (gamma / 2) * np.sum(w ** 2) + (beta / 2) * w0 ** 2

    total_loss = avg_log_loss + regularization
    return total_loss

#### 2. [0.5 Балл] Реализуйте функцию, которая будет возвращать градиенты весов вашей модели Логистической регрессии:

In [9]:
def get_grad(X: np.ndarray, y: np.ndarray, w: List[float], w0: float, gamma=1.0, beta=1.0) -> Tuple[List[float], float]:
    N = len(y)
    w = np.array(w)

    linear_comb = X.dot(w) + w0
    sigmoid = 1 / (1 + np.exp(-y * linear_comb))

    grad_w = -np.mean((1 - sigmoid) * y[:, np.newaxis] * X, axis=0) + gamma * w
    grad_w0 = -np.mean((1 - sigmoid) * y) + beta * w0

    return grad_w.tolist(), grad_w0

In [10]:
# код для проверки

np.random.seed(42)
X = np.random.multivariate_normal(np.arange(5), np.eye(5), size=10)
y = np.random.binomial(1, 0.42, size=10)
w, w0 = np.random.normal(size=5), np.random.normal()

grad_w, grad_w0 = get_grad(X, y, w, w0)
assert(np.allclose(grad_w,
                   [-2.73262076, -1.87176281, 1.30051144, 2.53598941, -2.71198109],
                   rtol=1e-2) & \
       np.allclose(grad_w0,
                   -0.2078231418067844,
                   rtol=1e-2)
)

ValueError: operands could not be broadcast together with shapes (10,10) (10,5) 

####  3. [1 Балл]  Реализуйте класс для модели логистической регрессии, используя выше написанные функции:

Модель должна обучаться методом SGD.

In [11]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import roc_curve

In [12]:
class Logit(BaseEstimator, ClassifierMixin):
    def __init__(self, beta=1.0, gamma=1.0, lr=1e-2, tolerance=1e-8, max_iter=1000, random_state=42):
        self.beta = beta
        self.gamma = gamma
        self.lr = lr
        self.tolerance = tolerance
        self.max_iter = max_iter
        self.random_state = random_state
        self.w = None
        self.w0 = None

    def fit(self, X: np.ndarray, y: np.ndarray):
        np.random.seed(self.random_state)
        y = 2 * y - 1

        n_features = X.shape[1]
        self.w = np.random.normal(size=n_features)
        self.w0 = np.random.normal()

        for i in range(self.max_iter):
            grad_w, grad_w0 = self.get_grad(X, y, self.w, self.w0, gamma=self.gamma, beta=self.beta)

            self.w -= self.lr * np.array(grad_w)
            self.w0 -= self.lr * grad_w0

            if np.linalg.norm(grad_w) < self.tolerance and abs(grad_w0) < self.tolerance:
                break

    def predict(self, X: np.ndarray) -> np.ndarray:
        probs = self.predict_proba(X)
        return (probs >= 0.5).astype(int)

    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        linear_comb = X.dot(self.w) + self.w0
        return 1 / (1 + np.exp(-linear_comb))

    def get_grad(self, X: np.ndarray, y: np.ndarray, w: List[float], w0: float, gamma=1.0, beta=1.0) -> Tuple[List[float], float]:
        N = len(y)
        linear_comb = X.dot(w) + w0
        sigmoid = 1 / (1 + np.exp(-y * linear_comb))

        grad_w = -np.mean((1 - sigmoid) * y[:, np.newaxis] * X, axis=0) + gamma * np.array(w)
        grad_w0 = -np.mean((1 - sigmoid) * y) + beta * w0

        return grad_w.tolist(), grad_w0


In [13]:
# этот код менять не надо!
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1800, n_features=2, n_redundant=0, n_informative=2,
                               random_state=42, n_clusters_per_class=1)
model = Logit(beta=1.0, gamma=1.0, lr=1e-2, tolerance=1e-8, max_iter=1000)
model.fit(X, y)

y_pred = model.predict(X)
y_proba = model.predict_proba(X)

print("Predicted Labels:", y_pred)
print("Predicted Probabilities:", y_proba[:5])

ValueError: operands could not be broadcast together with shapes (1800,1800) (1800,2) 

####  4. [0.5 Балл]  Реализуйте функцию, которая отрисовывает объекты вашего датасета, их метки и разделяющую гиперплоскость, полученную от Логистической регрессии (пример того, что должно получиться ниже):

In [ ]:
def plot_decision_boundary(model, X, y):
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))

    grid = np.c_[xx.ravel(), yy.ravel()]
    probs = model.predict_proba(grid).reshape(xx.shape)

    plt.contourf(xx, yy, probs, levels=[0, 0.5, 1], colors=['#FFAAAA', '#AAAAFF'], alpha=0.3)

    plt.scatter(X[y == -1][:, 0], X[y == -1][:, 1], color='red', label='Class -1', edgecolor='k')
    plt.scatter(X[y == 1][:, 0], X[y == 1][:, 1], color='blue', label='Class 1', edgecolor='k')

    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.legend()
    plt.title('Decision Boundary for Logistic Regression')
    plt.show()

In [ ]:
model = Logit(0,0)
y[y == 0] = -1
model.fit(X, y)
plot_decision_boundary(model, X, y)

#### 5. [0.5 Балл] Для предыдущей задачи отобразите на графике, как изменяется значение функция потерь от номера итерации.

In [ ]:
def plot_loss_history(model):
    plt.figure(figsize=(8, 6))
    plt.plot(model.loss_history, label="Loss over iterations")
    plt.xlabel("Iteration")
    plt.ylabel("Loss")
    plt.title("Loss vs. Iterations")
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
plot_loss_history(model)

#### 6. [2 Балл] Для данных, на которых тестировали модель Логистической регрессии, заиспользуйте модель SVC из библиотеки sklearn. Попробуйте различные ядра (kernel) и различные коэфициенты C. Посмотрите на метрики, которые мы обсуждали на занятии (Acc, Precision, Recall, AUC-ROC, F1-Score).

In [ ]:
rom sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

X, y = make_classification(n_samples=1800, n_features=2, n_redundant=0, n_informative=2,
                           random_state=42, n_clusters_per_class=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
kernels = ['linear', 'rbf', 'poly']
C_values = [0.1, 1, 10]

results = []
for kernel in kernels:
    for C in C_values:
        model = SVC(kernel=kernel, C=C, probability=True, random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1]

        acc = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        auc_roc = roc_auc_score(y_test, y_proba)

        results.append({
            'Kernel': kernel,
            'C': C,
            'Accuracy': acc,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1,
            'AUC-ROC': auc_roc
        })

results_df = pd.DataFrame(results)
print(results_df)

#### 7. [2 Балл] Реализуйте класс KNNClassifier, который должен реализовывать классификацию путем нахождения k ближайших соседей. В методе predict_proba Вам необходимо выдавать вектор вероятностей для каждого объекта, который означает, что объект является экземпляром i-го класса с p_i вероятностью. Протестируйте Ваш класс на данных, сгенерированных выше, посмотрите на метрики (Acc, Precision, Recall, AUC-ROC, F1-Score).

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(n_samples=1800, n_features=2, n_redundant=0, n_informative=2, random_state=42, n_clusters_per_class=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

knn_model = KNNClassifier(k=5)
knn_model.fit(X_train, y_train)

y_pred = knn_model.predict(X_test)
y_proba = knn_model.predict_proba(X_test)[:, 1]  # Вероятности для класса 1

acc = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_proba)

print(f"Accuracy: {acc}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")
print(f"AUC-ROC: {auc_roc}")


## **Практическая часть**

В этом задании мы будем работать с Датасетом Fashion Mnist. Это датасет, который представляет изображения одного канала с различными типами одежды. Вам необходимо провести полный пайплайн обучения моделей (KNN и Logreg), которые вы можете импортировать из библиотеки sklearn.

#### 8. [0 Балл] Импортируйте датафрейм из csv файла. Поделите выборку следующим образом - :50000 (Train) и 50000: (Test).

In [ ]:
df = pd.read_csv("train.csv", encoding='utf-8')
train_df = df.iloc[:50000]
test_df = df.iloc[50000:]

df

#### 9. [0.5 Балл] Визуализируйте некоторые из объектов датасета. В колонках отображены яркости пикселей, которые представляют из себя изображения Fashion Mnist. С помощью matplotlib визуализируйте по одному представителю каждого класса.

In [ ]:
unique_classes = df['Category'].unique()

# Создаем фигуру с подграфиками
plt.figure(figsize=(10, 10))

# Цикл по каждому классу
for i, category in enumerate(unique_classes):
    # Выбираем первый экземпляр данного класса
    class_sample = df[df['Category'] == category].iloc[0, 2:]  # Яркости пикселей, начиная с третьей колонки

    # Преобразуем яркости пикселей в 28x28 матрицу
    image = class_sample.values.reshape(28, 28)

    # Отображаем изображение на графике
    plt.subplot(4, 4, i + 1)  # Создаем сетку 4x4 для графиков
    plt.imshow(image, cmap='gray')
    plt.title(f'Class {category}')
    plt.axis('off')

plt.tight_layout()
plt.show()

#### 10. [0.5 Балл] Отнормируйте признаки в датасете, попробуйте два варианта StandartScaller и MinMaxScaller.

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
X = df.iloc[:, 2:]  # Признаки
y = df['Category']  # Метки классов

# Нормализация с помощью StandardScaler
standard_scaler = StandardScaler()
X_standard_scaled = standard_scaler.fit_transform(X)

# Нормализация с помощью MinMaxScaler
minmax_scaler = MinMaxScaler()
X_minmax_scaled = minmax_scaler.fit_transform(X)

# Преобразование обратно в DataFrame для наглядности (необязательно)
X_standard_scaled_df = pd.DataFrame(X_standard_scaled, columns=X.columns)
X_minmax_scaled_df = pd.DataFrame(X_minmax_scaled, columns=X.columns)

# Проверка
print("StandardScaler нормализованный датафрейм:\n", X_standard_scaled_df.head())
print("MinMaxScaler нормализованный датафрейм:\n", X_minmax_scaled_df.head())

#### 10. [2 Балл] Проведите эксперименты: для моделей KNeighborsClassifier и LogisticRegression подберите гиперпараметры с помощью GridSerchCV (минимум 5 фолдов). Получите качество моделей на тестовой выборке. Основная метрика в данном задании будет accuracy. Сравните эти две модели. Какая модель показывает лучшее качество, предположите почему и напишите ответ.

**NB!**: в задании нужно подбирать несколько гиперпараметров по сетке. Какие гиперпараметры подбирать - решаете Вы сами. Обязательно обоснуйте, почему и какие параметры Вы подбираете! Например, подбор только гиперпараметра C в LogisticRegression не будет засчитываться как решение данного задания! Попытайтесь серьезно отнестись к нему, будто вы за это получите зарплату 300к.

## **Бонусы**

#### Задача 1. [1 Балл] У Вас есть датасет с 10**4 объектами. У всех объектов два признака и все они одинаковые у всех объектов. Однако, 5000 - отрицательного класса и 5000 - положительного класса. Вы запускате Логистическую регрессию для классификации на данном датасете. Что Вы получите в итоге обучения данной модели на SGD? Ответ обоснуйте.

#### Задача 2. [1 Балл] При классификации Fashion Mnist модель Логистической регрессии на обучении многоклассовой классификации методом One-VS-All у Вас получилось k классификаторов. Изобразите веса ваших полученных моделей как изображения в matplotlib. Возможно, модель выучила какие-то графические паттерны в данных? Ответ обоснуйте.

#### Задача 3. [1 Балл] В задаче классификации Fashion Mnist Вы попытались выбить какой-то accuracy. Для получения бонусного балла Вам нужно на той же самой выборке получить значение метрики accuracy > 0.87 на тесте (Тестовую выборку менять нельзя, но обучающую можно). Какими моделями и методами Вы это будете делать - на Ваше усмотрение, но **нельзя использовать никакие нейронные сети**. Необходимо получить модель машинного обучения, выполняющую эту задачу.